# Pricing Options Using COS / FFT Methods of BS, Merton, Kou, and Heston

This workbook demonstrates the highly efficient pricing of European options using COS and FFT methods applied to the Black-Scholes (BS), Merton, Kou, and Heston models. It provides step-by-step guidance on how to calibrate these models, compute option prices across various strikes, and compare them to market-implied volatilities. The process is optimized for handling 0 DTE (zero days to expiry) options using high-frequency data.

## Loading 0DTE Option Data

The data loading process is handled by the `load_0dte_data` method, as demonstrated in the provided Python code. This method performs the following steps:

- Reads raw option data from CSV files using `pandas`.
- Processes timestamps and symbols to extract relevant information.

Although no explicit data preparation is required, this method sets up the foundation for accessing and analyzing the required market data for further processing.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os 

def load_0dte_data(hour='08'):
    """
    Load 0DTE option data for BTC and ETH from CSV files.
    The data is filtered to include only options that are expiring today.
    Returns a tuple of (btc_df, eth_df).
    """
    btc_df = pd.read_csv(os.path.join(f'/Users/joris/Documents/Master QF/Thesis/optimal-gamma-hedging/Data/calibration_data/{hour}', f'btc_{hour}_0dte_data.csv'))
    eth_df = pd.read_csv(os.path.join(f'/Users/joris/Documents/Master QF/Thesis/optimal-gamma-hedging/Data/calibration_data/{hour}', f'eth_{hour}_0dte_data.csv'))

    # Convert the timestamps to UTC 
    btc_df['timestamp'] = pd.to_datetime(btc_df['timestamp'], unit='us', utc=True)
    eth_df['timestamp'] = pd.to_datetime(eth_df['timestamp'], unit='us', utc=True)
    return btc_df, eth_df


## Black-Scholes Functions:
The code includes functions to calculate Black-Scholes option prices for both call and put options. Additionally, the `implied_volatility` function is used to compute the volatility that corresponds to an observed market option price. It does this using a numerical solver to match the market price to the Black-Scholes formula. This step is critical for model calibration, as it enables the comparison of implied volatilities generated by the model to those observed in the market.


In [ ]:
import scipy.stats as st
import scipy.optimize as optimize

def extract_inputs_from_df(df):
    return (
        df['opt_type'].values,
        df['spot'].values,
        df['strike'].values,        
        df['time_to_maturity'].values,
        df['market_price'].values
        )

def bs_call_put_prices(CP,S_0,K,sigma,tau,r):
    # Get Call and Put Option prices
    d1    = (np.log(S_0 / K) + (r + 0.5 * np.power(sigma,2.0)) * (tau)) / (sigma * np.sqrt(tau))
    d2    = d1 - sigma * np.sqrt(tau)


    value = np.where(CP == 'call', st.norm.cdf(d1) * S_0 - st.norm.cdf(d2) * K * np.exp(-r * (tau)),
                        st.norm.cdf(-d2) * K * np.exp(-r * (tau)) - st.norm.cdf(-d1)*S_0)
    return value

def bs_delta(CP,S_0,K,sigma,tau,r):
    # Get BS Delta values
    d1    = (np.log(S_0 / K) + (r + 0.5 * np.power(sigma,2.0)) * (tau)) / (sigma * np.sqrt(tau))
    value = np.where(CP == 'call', st.norm.cdf(d1), st.norm.cdf(d1) - 1.0)
    return value

def bs_impliedvol(CP,marketPrice,K,tau,S_0,r):
    func = lambda sigma: np.power(bs_call_put_prices(CP,S_0,K,sigma,tau,r) - marketPrice, 1.0)
    impliedVol = optimize.newton(func, 0.7, tol=1e-9)
    #impliedVol = optimize.brent(func, brack= (0.05, 2))
    return impliedVol

# Load the 0DTE data for BTC and ETH at a specific hour
df_btc, df_eth = load_0dte_data(hour='08')
r = 0.05  # Risk-free rate, can be adjusted as needed
sigma = 0.7  # Initial guess for volatility, can be adjusted as needed

# Move on with BTC only for now
df = df_btc
CP, S_0, K, tau, mark_price = extract_inputs_from_df(df)
bs_prices = bs_call_put_prices(CP, S_0, K, sigma, tau, r)

## Characteristic Functions for Pricing Models:
- **Merton Jump-Diffusion Model:** The `char_func_merton` function implements the characteristic function for the Merton jump-diffusion model, which accounts for both continuous diffusion and occasional jumps in the asset price process. This is useful for capturing sudden, large price movements in the market.
- **Kou Jump-Diffusion Model:** The `char_func_kou` function is based on the Kou model, where jump sizes follow a double exponential distribution. This allows for more flexibility in modeling both upward and downward jumps, which is often more realistic for financial markets than the normal distribution assumed in Merton’s model.
- **Heston Stochastic Volatility Model:** The `char_func_heston` function models the characteristic function for the Heston model, where volatility itself is stochastic, meaning it follows a random process. This model captures more complex market dynamics, such as volatility smiles or skew, and is often used in markets with high volatility or large price fluctuations.


In [ ]:
import numpy as np

i = 1j    # imag unit
SEC_PER_YEAR = 365 * 24 * 3600 # seconds to years

# ------------------  MERTON  ------------------
def phi_merton(u, sigma, xi, muJ, sigmaJ, r, tau_sec):
    tau = tau_sec / SEC_PER_YEAR
    omega_bar = xi * (np.exp(muJ + 0.5*sigmaJ**2) - 1)
    mu = r - 0.5*sigma**2 - omega_bar
    return np.exp(i*u*mu*tau -0.5*sigma**2*u**2*tau +
                  xi*tau*(np.exp(i*muJ*u - 0.5*sigmaJ**2*u**2) - 1))

def chf_merton(u, sigma, xi, muJ, sigmaJ, r, tau_sec, S0):
    return np.exp(i*u*np.log(S0)) * phi_merton(u, sigma, xi, muJ, sigmaJ, r, tau_sec)

# ------------------  KOU  ---------------------
def phi_kou(u, sigma, xi, p1, alpha1, alpha2, r, tau_sec):
    p2 = 1 - p1
    tau = tau_sec / SEC_PER_YEAR
    omega_bar = xi * (p1*alpha1/(alpha1-1) + (1-p1)*alpha2/(alpha2+1) - 1)
    mu = r - 0.5*sigma**2 - omega_bar
    return np.exp(i*u*mu*tau-0.5*sigma**2*u**2*tau +
                  xi*tau*((p1*alpha1)/(alpha1 - i*u) +
                          (p2*alpha2)/(alpha2 + i*u) - 1))

def chf_kou(u, sigma, xi, p1, alpha1, alpha2, r, tau_sec, S0):
    return np.exp(i*u*np.log(S0)) * phi_kou(u, sigma, xi, p1, alpha1, alpha2, r, tau_sec)

# ------------------  HESTON  ------------------
def phi_heston(u, tau, r, kappa, v_bar, gamma, rho, v0):
    d1 = np.sqrt((kappa - i*rho*gamma*u)**2 + (u**2 + i*u)*gamma**2)
    g  = (kappa - i*rho*gamma*u - d1) / (kappa - i*rho*gamma*u + d1)
    term_r  = i*u*r*tau
    term_v0 = (v0 / gamma**2) * ((1 - np.exp(-d1*tau)) /
                                 (1 - g*np.exp(-d1*tau))) * (kappa - i*rho*gamma*u - d1)
    term_bar= (kappa*v_bar / gamma**2) * (tau*(kappa - i*rho*gamma*u - d1) -
                                          2*np.log((1 - g*np.exp(-d1*tau))/(1 - g)))
    return np.exp(term_r + term_v0 + term_bar)

def chf_heston(u, kappa, v_bar, gamma, rho, v0, r, tau_sec, S0):
    tau = tau_sec / SEC_PER_YEAR
    return np.exp(i*u*np.log(S0)) * phi_heston(u, tau, r, kappa, v_bar, gamma, rho, v0)


## COS Valuation Method (`cos_valuation`):
The `cos_valuation` method is the core pricing engine in the code. It uses the COS method, a numerical technique that approximates the option price by expanding the characteristic function into a cosine series. This method is efficient and accurate, especially for pricing European options across a range of strikes. The method takes the characteristic function from one of the models, the spot price, strike price, risk-free rate, time to maturity, and the option type (call or put) as inputs. The integration range `[a, b]` is determined based on the cumulants of the log-return distribution, ensuring accuracy in pricing.


## RMSE Loss Function (`calculate_rmse_iv`):
The `calculate_rmse_iv` function is crucial for calibrating the model parameters. It computes the Root Mean Squared Error (RMSE) between the implied volatilities generated by the model and the actual market-implied volatilities. The function calculates theoretical prices using the `cos_valuation` method, converts these prices into model-implied volatilities, and compares them to the market values. The objective is to minimize this RMSE during the calibration process, ensuring the model parameters provide the best fit to the market data.


## Parameter Calibration Methods (`calibrate_merton`, `calibrate_kou`, `calibrate_heston`):
These methods are responsible for calibrating the model parameters to match market data. The calibration process involves two steps:
- **Global Search:** A global search is performed using `scipy.optimize.differential_evolution`, which explores the parameter space to find a promising region where the optimal parameters are likely to be.
- **Local Refinement:** Once a promising region is found, local refinement is done using `scipy.optimize.minimize` with the BFGS algorithm, which fine-tunes the parameters to minimize the RMSE.

These methods iteratively adjust the model parameters, ensuring that the model’s implied volatilities closely match the observed market-implied volatilities. Predefined bounds for each model parameter ensure that the optimizer searches within realistic and valid ranges.
